In [1]:
import sys
# sys.path is a list of absolute path strings
sys.path.append('/mnt/sda/boris_stanoev/PropStar/')

In [2]:
from normalization.normalize import Normalize
from sklearn.datasets import fetch_openml

In [3]:
data = fetch_openml(name='telco-customer-churn')

/mnt/sda/boris_stanoev/PropStar/venv/lib/python3.10/site-packages/sklearn/datasets/_openml.py:303: UserWarning: Multiple active versions of the dataset matching the name telco-customer-churn exist. Versions may be fundamentally different, returning version 1.
  warn(
/mnt/sda/boris_stanoev/PropStar/venv/lib/python3.10/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [4]:
drop_indices = data.data[data.data['TotalCharges']== ' '].index
data.data.drop(labels=[*drop_indices], inplace=True)
data.target.drop(labels=[*drop_indices], inplace=True)


/tmp/ipykernel_556813/1379662505.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.data.drop(labels=[*drop_indices], inplace=True)


In [5]:
len(data.data)

7032

In [6]:
denormalized_table = data.data.join(data.target)

- Customers who left within the last month – the column is called Churn
- 
Services that each customer has signed up for – phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movie
- 
Customer account informati:  – how long they’ve been a customer, contract, payment method, paperless billing, monthly charges, and total charg
- s
Demographic info about custom:s – gender, age range, and if they have partners and dependents

In [7]:
denormalized_table.columns.values

array(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges',
       'TotalCharges', 'Churn'], dtype=object)

In [8]:
denormalized_table[['SeniorCitizen']].groupby(['SeniorCitizen'])['SeniorCitizen'].count()

SeniorCitizen
0    5890
1    1142
Name: SeniorCitizen, dtype: int64

In [9]:
all_columns = denormalized_table.columns.values
demographics_columns = ['gender', 'SeniorCitizen', 'Partner', 'Dependents']
customer_columns = ['Contract',
       'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges',
       'TotalCharges', 'tenure', 'Churn']
service_columns = ['PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies']
# churn_target = ['Churn']

In [10]:
print(denormalized_table[[*demographics_columns]].count())
print(denormalized_table[[*demographics_columns]].drop_duplicates().count())

gender           7032
SeniorCitizen    7032
Partner          7032
Dependents       7032
dtype: int64
gender           16
SeniorCitizen    16
Partner          16
Dependents       16
dtype: int64


In [11]:
print(denormalized_table[[*customer_columns]].count())
print(denormalized_table[[*customer_columns]].drop_duplicates().count())

Contract            7032
PaperlessBilling    7032
PaymentMethod       7032
MonthlyCharges      7032
TotalCharges        7032
tenure              7032
Churn               7032
dtype: int64
Contract            6952
PaperlessBilling    6952
PaymentMethod       6952
MonthlyCharges      6952
TotalCharges        6952
tenure              6952
Churn               6952
dtype: int64


In [12]:
print(denormalized_table[[*service_columns]].count())
print(denormalized_table[[*service_columns]].drop_duplicates().count())

PhoneService        7032
MultipleLines       7032
InternetService     7032
OnlineSecurity      7032
OnlineBackup        7032
DeviceProtection    7032
TechSupport         7032
StreamingTV         7032
StreamingMovies     7032
dtype: int64
PhoneService        322
MultipleLines       322
InternetService     322
OnlineSecurity      322
OnlineBackup        322
DeviceProtection    322
TechSupport         322
StreamingTV         322
StreamingMovies     322
dtype: int64


In [13]:
norm = Normalize(denormalized_table=denormalized_table, denormalized_table_name="original_table")

In [14]:
norm.create_entity(source_entity_name="original_table", target_entity_name="customer", columns=all_columns,
                   distinct_values=True, retain_columns=['customer_id'])

In [15]:
norm.create_entity(source_entity_name="customer", target_entity_name="demographics", columns=demographics_columns,
                   distinct_values=True, retain_columns=['customer_id'])

In [16]:
norm.create_entity(source_entity_name="customer", target_entity_name="service", columns=service_columns,
                   distinct_values=True, retain_columns=['customer_id'])

In [17]:
norm.create_entity(source_entity_name="customer", target_entity_name="contract_type", columns=['Contract'],
                   distinct_values=True, retain_columns=['customer_id'])

In [18]:
norm.create_entity(source_entity_name="customer", target_entity_name="payment_method_type", columns=['PaymentMethod'],
                   distinct_values=True, retain_columns=['customer_id'])

In [19]:
norm.remove_entity("original_table")

In [20]:
print(norm.primary_keys)
print("\n")
print(norm.relations)
print("\n")
print(norm.entity_set.keys())


{'customer': 'customer_id', 'demographics': 'demographics_id', 'service': 'service_id', 'contract_type': 'contract_type_id', 'payment_method_type': 'payment_method_type_id'}


{'customer': {}, 'demographics': {'customer': ('demographics_id', 'demographics_id')}, 'service': {'customer': ('service_id', 'service_id')}, 'contract_type': {'customer': ('contract_type_id', 'contract_type_id')}, 'payment_method_type': {'customer': ('payment_method_type_id', 'payment_method_type_id')}}


dict_keys(['customer', 'demographics', 'service', 'contract_type', 'payment_method_type'])


In [21]:
for entity in norm.entity_set.keys():
    print(
        f"ALTER TABLE {entity} ADD CONSTRAINT PK_{entity} PRIMARY KEY ({norm.primary_keys[entity]});")

for parent_table in norm.relations:
    for child_table in norm.relations[parent_table]:
        print(f"ALTER TABLE {child_table} ADD CONSTRAINT FK_{child_table}_{parent_table} "
              f"FOREIGN KEY ({norm.relations[parent_table][child_table][1]}) "
              f"REFERENCES {parent_table}({norm.relations[parent_table][child_table][0]}));")


ALTER TABLE customer ADD CONSTRAINT PK_customer PRIMARY KEY (customer_id);
ALTER TABLE demographics ADD CONSTRAINT PK_demographics PRIMARY KEY (demographics_id);
ALTER TABLE service ADD CONSTRAINT PK_service PRIMARY KEY (service_id);
ALTER TABLE contract_type ADD CONSTRAINT PK_contract_type PRIMARY KEY (contract_type_id);
ALTER TABLE payment_method_type ADD CONSTRAINT PK_payment_method_type PRIMARY KEY (payment_method_type_id);
ALTER TABLE customer ADD CONSTRAINT FK_customer_demographics FOREIGN KEY (demographics_id) REFERENCES demographics(demographics_id));
ALTER TABLE customer ADD CONSTRAINT FK_customer_service FOREIGN KEY (service_id) REFERENCES service(service_id));
ALTER TABLE customer ADD CONSTRAINT FK_customer_contract_type FOREIGN KEY (contract_type_id) REFERENCES contract_type(contract_type_id));
ALTER TABLE customer ADD CONSTRAINT FK_customer_payment_method_type FOREIGN KEY (payment_method_type_id) REFERENCES payment_method_type(payment_method_type_id));


In [22]:
norm.persist_entity_set(database_name="telco_churn_exp1")

2023-10-03 17:06:07,795 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2023-10-03 17:06:07,795 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-03 17:06:07,796 INFO sqlalchemy.engine.Engine SELECT schema_name()
2023-10-03 17:06:07,796 INFO sqlalchemy.engine.Engine [generated in 0.00040s] ()
2023-10-03 17:06:07,797 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2023-10-03 17:06:07,798 INFO sqlalchemy.engine.Engine [generated in 0.00041s] ()
2023-10-03 17:06:07,799 INFO sqlalchemy.engine.Engine SELECT 1 FROM fn_listextendedproperty(default, default, default, default, default, default, default)
2023-10-03 17:06:07,799 INFO sqlalchemy.engine.Engine [generated in 0.00030s] ()
2023-10-03 17:06:07,800 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-03 17:06:07,800 INFO sqlalchemy.engine.Engine IF (SCHEMA_ID('telco_churn_exp1') IS NOT NULL) BEGIN DROP SCHEMA [telco_churn_exp1]; END 
2023-10-03 17:06:07,800 INFO sq